## Step 1: Load the MIMIC-III Dataset

We'll load the MIMIC-III dataset using PyHealth 2's new `MIMIC3Dataset` class. We need to specify:
- `root`: Path to the MIMIC-III data directory
- `tables`: Clinical tables to load (diagnoses, procedures, prescriptions)
- `dev`: Set to `True` for development/testing with a small subset of data

In [1]:
from pyhealth.datasets import MIMIC3Dataset

# Load MIMIC-III dataset
dataset = MIMIC3Dataset(
    root=r"F:\coding_projects\pyhealth\downloads\mimic-iii-demo",
    tables=["diagnoses_icd", "procedures_icd", "prescriptions"],
    dev=True,  # Set to False for full dataset
)

dataset.stats()

No config path provided, using default config
Initializing mimic3 dataset from F:\coding_projects\pyhealth\downloads\mimic-iii-demo (dev mode: True)
Initializing mimic3 dataset from F:\coding_projects\pyhealth\downloads\mimic-iii-demo (dev mode: True)
Scanning table: patients from F:\coding_projects\pyhealth\downloads\mimic-iii-demo\PATIENTS.csv.gz
Original path does not exist. Using alternative: F:\coding_projects\pyhealth\downloads\mimic-iii-demo\PATIENTS.csv
Scanning table: admissions from F:\coding_projects\pyhealth\downloads\mimic-iii-demo\ADMISSIONS.csv.gz
Original path does not exist. Using alternative: F:\coding_projects\pyhealth\downloads\mimic-iii-demo\ADMISSIONS.csv
Scanning table: icustays from F:\coding_projects\pyhealth\downloads\mimic-iii-demo\ICUSTAYS.csv.gz
Original path does not exist. Using alternative: F:\coding_projects\pyhealth\downloads\mimic-iii-demo\ICUSTAYS.csv
Scanning table: patients from F:\coding_projects\pyhealth\downloads\mimic-iii-demo\PATIENTS.csv.gz
O

F:\coding_projects\pyhealth\pyhealth\datasets\mimic3.py:50: UserWarning: Events from prescriptions table only have date timestamp (no specific time). This may affect temporal ordering of events.
  warnings.warn(


## Step 2: Define the Mortality Prediction Task

PyHealth 2 uses task classes to define how to extract samples from the raw EHR data. The `MortalityPredictionMIMIC3` task:
- Extracts diagnosis codes (ICD-9), procedure codes, and drug information from each visit
- Creates binary labels based on mortality in the next visit
- Filters out visits without sufficient clinical codes

You can optionally specify a `cache_dir` to save processed samples for faster future loading.

In [2]:
from pyhealth.tasks import MortalityPredictionMIMIC3

# Define the mortality prediction task
task = MortalityPredictionMIMIC3()

# Apply the task to generate samples
samples = dataset.set_task(
    task=task,
    cache_dir="./cache_mortality_mimic3"  # Cache processed samples
)

print(f"Generated {len(samples)} samples")
print(f"\nInput schema: {samples.input_schema}")
print(f"Output schema: {samples.output_schema}")

Setting task MortalityPredictionMIMIC3 for mimic3 base dataset...
Loading cached samples from cache_mortality_mimic3\MortalityPredictionMIMIC3.parquet
Loading cached samples from cache_mortality_mimic3\MortalityPredictionMIMIC3.parquet
Loaded 26 cached samples
Label mortality vocab: {0: 0, 1: 1}
Loaded 26 cached samples
Label mortality vocab: {0: 0, 1: 1}


Processing samples: 100%|██████████| 26/26 [00:00<00:00, 5198.89it/s]

Generated 26 samples for task MortalityPredictionMIMIC3
Generated 26 samples

Input schema: {'conditions': 'sequence', 'procedures': 'sequence', 'drugs': 'sequence'}
Output schema: {'mortality': 'binary'}
Generated 26 samples

Input schema: {'conditions': 'sequence', 'procedures': 'sequence', 'drugs': 'sequence'}
Output schema: {'mortality': 'binary'}


## Step 3: Explore a Sample

Let's examine what a single sample looks like. Each sample represents one hospital visit with:
- **conditions**: List of ICD-9 diagnosis codes
- **procedures**: List of ICD-9 procedure codes  
- **drugs**: List of drug names
- **mortality**: Binary label (0 = survived, 1 = deceased in next visit)

In [3]:
# Display a sample
print("Sample structure:")
print(samples[0])

# Show statistics
print("\n" + "="*50)
print("Dataset Statistics:")
print("="*50)

# Count unique codes
all_conditions = set()
all_procedures = set()
all_drugs = set()
mortality_count = 0
for sample in samples:
    all_conditions.update(sample['conditions'])
    all_procedures.update(sample['procedures'])
    all_drugs.update(sample['drugs'])
    mortality_count += float(sample['mortality'])

print(f"Unique diagnosis codes: {len(all_conditions)}")
print(f"Unique procedure codes: {len(all_procedures)}")
print(f"Unique drugs: {len(all_drugs)}")
print(f"\nMortality rate: {mortality_count/len(samples)*100:.2f}%")
print(f"Positive samples: {mortality_count}")
print(f"Negative samples: {len(samples) - mortality_count}")

Sample structure:
{'hadm_id': '102203', 'patient_id': '42135', 'conditions': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), 'procedures': tensor([1, 2]), 'drugs': tensor([ 1,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  3, 14, 15, 16,
        17, 16, 17,  1, 17, 18,  4, 19, 18, 19, 20,  1, 18, 21, 22, 18, 18, 21,
        14,  1,  1, 21, 18, 18, 14, 23, 23,  1, 24, 25, 19,  3,  3, 26,  2,  3,
        23,  4, 13, 12, 11,  9,  8,  6, 23, 10, 14, 27, 28, 28, 28, 29,  1, 30,
        31, 32, 33,  3,  3,  5,  7, 23,  9, 11,  4,  6,  8, 12, 13, 15, 18, 18,
        14, 34, 18, 18, 20,  3, 14]), 'mortality': tensor([1.])}

Dataset Statistics:
Unique diagnosis codes: 404
Unique procedure codes: 101
Unique drugs: 2647

Mortality rate: 19.23%
Positive samples: 5.0
Negative samples: 21.0


## Step 4: Split the Dataset

We split the data into training, validation, and test sets using a 70-10-20 split.

**Note:** We use `split_by_sample` which randomly splits samples. For time-series tasks, you might want to use temporal splits to avoid data leakage.

In [4]:
from pyhealth.datasets import split_by_sample

# Split dataset: 70% train, 10% validation, 20% test
train_dataset, val_dataset, test_dataset = split_by_sample(
    dataset=samples, 
    ratios=[0.7, 0.1, 0.2]
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Training samples: 18
Validation samples: 2
Test samples: 6


## Step 5: Create Data Loaders

Data loaders batch the samples and handle data feeding during training and evaluation.

In [5]:
from pyhealth.datasets import get_dataloader

# Create data loaders
train_dataloader = get_dataloader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=2, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=2, shuffle=False)

print(f"Training batches: {len(train_dataloader)}")
print(f"Validation batches: {len(val_dataloader)}")
print(f"Test batches: {len(test_dataloader)}")

Training batches: 9
Validation batches: 1
Test batches: 3

Validation batches: 1
Test batches: 3


## Step 6: Initialize the AdaCare Model

The AdaCare model in PyHealth 2 automatically handles different feature types:
- **Sequence features** (like diagnosis/procedure/drug codes) are embedded using learned embeddings
- **Multiple feature keys** are processed by separate AdaCare layers
- The model provides interpretability through attention weights

### Key Parameters:
- `embedding_dim`: Dimension of code embeddings (default: 128)
- `hidden_dim`: Hidden dimension of GRU layers (default: 128)
- `kernel_size`: Kernel size for causal convolution (default: 2)
- `kernel_num`: Number of convolution kernels (default: 64)
- `dropout`: Dropout rate for regularization (default: 0.5)

In [6]:
from pyhealth.models import AdaCare

# Initialize AdaCare model
model = AdaCare(
    dataset=samples,
    embedding_dim=128,
    hidden_dim=128,
)

print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
print(f"\nModel architecture:")
print(model)

f:\coding_projects\pyhealth\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model initialized with 816369 parameters

Model architecture:
AdaCare(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict(
    (conditions): Embedding(191, 128, padding_idx=0)
    (procedures): Embedding(46, 128, padding_idx=0)
    (drugs): Embedding(298, 128, padding_idx=0)
  ))
  (adacare): ModuleDict(
    (conditions): AdaCareLayer(
      (nn_conv1): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(1,))
      (nn_conv3): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(3,), dilation=(3,))
      (nn_conv5): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(5,), dilation=(5,))
      (nn_convse): Recalibration(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (nn_c): Linear(in_features=192, out_features=48, bias=True)
        (nn_rescale): Linear(in_features=48, out_features=192, bias=True)
        (sparsemax): Sparsemax()
        (softmax): Softmax(dim=1)
      )
      (nn_inputse): Recalibration(
        (avg_pool): Adapt

## Step 7: Train the Model

We use PyHealth's `Trainer` class which handles:
- Training loop with automatic batching
- Validation during training
- Model checkpointing based on validation metrics
- Early stopping

We monitor the **ROC-AUC** score on the validation set.

In [7]:
from pyhealth.trainer import Trainer

# Initialize trainer
trainer = Trainer(
    model=model,
    metrics=["roc_auc", "pr_auc", "accuracy", "f1"]  # Track multiple metrics
)

# Train the model
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=50,
    monitor="roc_auc",  # Use ROC-AUC for model selection
    optimizer_params={"lr": 1e-3},  # Learning rate
)

AdaCare(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict(
    (conditions): Embedding(191, 128, padding_idx=0)
    (procedures): Embedding(46, 128, padding_idx=0)
    (drugs): Embedding(298, 128, padding_idx=0)
  ))
  (adacare): ModuleDict(
    (conditions): AdaCareLayer(
      (nn_conv1): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(1,))
      (nn_conv3): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(3,), dilation=(3,))
      (nn_conv5): CausalConv1d(128, 64, kernel_size=(2,), stride=(1,), padding=(5,), dilation=(5,))
      (nn_convse): Recalibration(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (nn_c): Linear(in_features=192, out_features=48, bias=True)
        (nn_rescale): Linear(in_features=48, out_features=192, bias=True)
        (sparsemax): Sparsemax()
        (softmax): Softmax(dim=1)
      )
      (nn_inputse): Recalibration(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (nn_c): Linear(in_features

Epoch 0 / 50: 100%|██████████| 9/9 [00:00<00:00, 20.61it/s]

--- Train epoch-0, step-9 ---
loss: 0.4599
loss: 0.4599



Evaluation: 100%|██████████| 1/1 [00:00<00:00, 99.97it/s]

--- Eval epoch-0, step-9 ---
roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
f1: 0.0000
loss: 0.7692
New best roc_auc score (1.0000) at epoch-0, step-9
f1: 0.0000
loss: 0.7692
New best roc_auc score (1.0000) at epoch-0, step-9




...

--- Eval epoch-48, step-441 ---
roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
f1: 0.0000
loss: 2.3073

roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
f1: 0.0000
loss: 2.3073




Epoch 49 / 50: 100%|██████████| 9/9 [00:00<00:00, 26.86it/s]

--- Train epoch-49, step-450 ---
loss: 0.0001
loss: 0.0001



Evaluation: 100%|██████████| 1/1 [00:00<00:00, 111.12it/s]

--- Eval epoch-49, step-450 ---
roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
f1: 0.0000
loss: 2.3182
Loaded best model
roc_auc: 1.0000
pr_auc: 1.0000
accuracy: 0.5000
f1: 0.0000
loss: 2.3182
Loaded best model


## Step 8: Evaluate on Test Set

After training, we evaluate the model on the held-out test set to measure its generalization performance.

In [8]:
# Evaluate on test set
test_results = trainer.evaluate(test_dataloader)

print("\n" + "="*50)
print("Test Set Performance")
print("="*50)
for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")

Evaluation: 100%|██████████| 3/3 [00:00<00:00, 50.40it/s]


Test Set Performance
roc_auc: 0.1250
pr_auc: 0.2917
accuracy: 0.6667
f1: 0.0000
loss: 0.7619


## Step 9: Model Interpretability (Optional)

One of AdaCare's key features is interpretability. The model provides attention weights that indicate which features are most important for predictions.

Let's examine the feature importance for a few test samples.

In [9]:
import torch
import numpy as np

# Get a batch from test set
test_batch = next(iter(test_dataloader))

# Run model in evaluation mode
model.eval()
with torch.no_grad():
    output = model(**test_batch)

# Extract interpretability information
if 'feature_importance' in output:
    print("Feature importance available!")
    print(f"Shape: {output['feature_importance']}")
    
    # Display importance for first sample
    print("\nFeature importance for first sample:")
    print("This shows which clinical features the model focuses on.")
else:
    print("Feature importance not available in model output.")

# Display predictions
print("\n" + "="*50)
print("Sample Predictions:")
print("="*50)
predictions = output['y_prob'].cpu().numpy()
true_labels = output['y_true'].cpu().numpy()

for i in range(min(5, len(predictions))):
    pred = predictions[i][0]
    true = int(true_labels[i][0])
    print(f"Sample {i+1}: Predicted={pred:.3f}, True={true}, Prediction={'Mortality' if pred > 0.5 else 'Survival'}")

Feature importance available!
Shape: [tensor([[[0.3808, 0.5472, 0.4767,  ..., 0.5502, 0.5235, 0.5840],
         [0.5611, 0.5604, 0.4398,  ..., 0.5486, 0.5007, 0.5026],
         [0.4841, 0.5169, 0.4600,  ..., 0.4207, 0.4374, 0.5361],
         ...,
         [0.4293, 0.5795, 0.3993,  ..., 0.5154, 0.5159, 0.5424],
         [0.4928, 0.4697, 0.4777,  ..., 0.5893, 0.4837, 0.5156],
         [0.5197, 0.5583, 0.4666,  ..., 0.4400, 0.4263, 0.5351]],

        [[0.5389, 0.5437, 0.4693,  ..., 0.5332, 0.4713, 0.5942],
         [0.4667, 0.5731, 0.5536,  ..., 0.4723, 0.4179, 0.6412],
         [0.4448, 0.5027, 0.4800,  ..., 0.4831, 0.4847, 0.5509],
         ...,
         [0.5170, 0.4971, 0.4677,  ..., 0.5119, 0.4715, 0.5198],
         [0.5170, 0.4971, 0.4677,  ..., 0.5119, 0.4715, 0.5198],
         [0.5170, 0.4971, 0.4677,  ..., 0.5119, 0.4715, 0.5198]]]), tensor([[[0.4630, 0.5156, 0.5087,  ..., 0.4908, 0.4307, 0.5002],
         [0.5030, 0.5333, 0.5260,  ..., 0.4907, 0.4975, 0.5351],
         [0.5030, 0